# CS4001/4042 Assignment 1, Part B, Q2
In Question B1, we used the Category Embedding model. This creates a feedforward neural network in which the categorical features get learnable embeddings. In this question, we will make use of a library called Pytorch-WideDeep. This library makes it easy to work with multimodal deep-learning problems combining images, text, and tables. We will just be utilizing the deeptabular component of this library through the TabMlp network:

In [ ]:
!pip install pytorch-widedeep

In [ ]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.models import TabMlp, WideDeep
from pytorch_widedeep import Trainer
from pytorch_widedeep.metrics import R2Score

import torch
from pytorch_widedeep.models import Wide
import math
from sklearn.metrics import mean_squared_error, r2_score


>Divide the dataset (‘hdb_price_prediction.csv’) into train and test sets by using entries from the year 2020 and before as training data, and entries from 2021 and after as the test data.

In [ ]:
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here
train_df = df[df['year'] <= 2020]
test_df = df[df['year'] >= 2021]

In [ ]:
train = train_df.to_numpy()
test = test_df.to_numpy()

>Refer to the documentation of Pytorch-WideDeep and perform the following tasks:
https://pytorch-widedeep.readthedocs.io/en/latest/index.html
* Use [**TabPreprocessor**](https://pytorch-widedeep.readthedocs.io/en/latest/examples/01_preprocessors_and_utils.html#2-tabpreprocessor) to create the deeptabular component using the continuous
features and the categorical features. Use this component to transform the training dataset.
* Create the [**TabMlp**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/model_components.html#pytorch_widedeep.models.tabular.mlp.tab_mlp.TabMlp) model with 2 linear layers in the MLP, with 200 and 100 neurons respectively.
* Create a [**Trainer**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/trainer.html#pytorch_widedeep.training.Trainer) for the training of the created TabMlp model with the root mean squared error (RMSE) cost function. Train the model for 100 epochs using this trainer, keeping a batch size of 64. (Note: set the *num_workers* parameter to 0.)

In [ ]:
# TODO: Enter your code here

continuous_cols = ["dist_to_nearest_stn", "dist_to_dhoby", "degree_centrality", "eigenvector_centrality", "remaining_lease_years", "floor_area_sqm"]
cat_embed_cols = ["month", "town", "flat_model_type", "storey_range"]
target = train_df["resale_price"].values

tab_preprocessor = TabPreprocessor(
    cat_embed_cols = cat_embed_cols, 
		continuous_cols = continuous_cols
)
X_tab = tab_preprocessor.fit_transform(train_df)

# tab_preprocessor.column_idx is a dictionary{key=column_name: value=index of column}
# tab_preprocessor.cat_embed_input is 
# [('month', 12, 6),
#  ('town', 26, 10),
#  ('flat_model_type', 43, 13),
#  ('storey_range', 17, 8)]
tabmlp = TabMlp(mlp_hidden_dims=[200,100], column_idx=tab_preprocessor.column_idx, cat_embed_input= tab_preprocessor.cat_embed_input, cat_embed_activation=None, continuous_cols=continuous_cols)

model = WideDeep(deeptabular=tabmlp)

trainer = Trainer(model, objective="root_mean_squared_error", num_workers=0)
trainer.fit(X_tab=X_tab, target=target, n_epochs=100, batch_size=64)

>Report the test RMSE and the test R2 value that you obtained.

In [ ]:
# TODO: Enter your code here

X_tab_test = tab_preprocessor.transform(test_df)
preds = trainer.predict(X_tab=X_tab_test)

In [ ]:
mse = mean_squared_error(y_true = list(test_df['resale_price']), y_pred = preds)
rsme = math.sqrt(mse)
r2 = r2_score(y_true = list(test_df['resale_price']), y_pred = preds)

print("Root Mean Squared Error: ", rsme)
print("R2 Score: ", r2)